# Exploratory study of the REF 2021 submission data

In [6]:
import os
import sys
import pandas as pd

sys.path.append('../src/')
import codebook as cb
import read_write as rw
import visualisations as vis

In [7]:
fname = rw.DATA_PPROC_IMPACTS
dset = pd.read_csv(os.path.join(rw.PROJECT_PATH, fname),
                   index_col=None,
                   dtype={0: str})
print(f"Read {fname}: {dset.shape[0]} records")

# for column in [cb.COL_OUTPUT_TYPE_NAME, cb.COL_OPEN_ACCESS]:
#     vis.calculate_and_visualise_counts(dset, column)

# select and save the records that contain software-related terms defined by root in title
dset_selected = pd.DataFrame()
for column in [cb.COL_IMPACT_TITLE, cb.COL_IMPACT_SUMMARY]:
    for term in cb.TERMS_SOFTWARE_RELATED_ROOTS:
        dset_selected = dset_selected.append(dset[dset[column].str.contains(term)])
    dset_selected = dset_selected.drop_duplicates(subset=[column])
print(f"Selected {dset_selected.shape[0]} records")



Read data/processed/ImpactCaseStudies_ppreprocessed.csv.gz: 6362 records
Selected 74 records


In [3]:
print(dset.columns)

Index(['Institution UKPRN code', 'Institution name', 'Main panel',
       'Unit of assessment number', 'Unit of assessment name',
       'Multiple submission letter', 'Multiple submission name',
       'Joint submission', 'Title', 'Is continued from 2014', 'Countries',
       'Formal partners', 'Funding programmes', 'Global research identifiers',
       'Name of funders', 'Researcher ORCIDs', 'Grant funding',
       '1. Summary of the impact', '2. Underpinning research',
       '3. References to the research', '4. Details of the impact',
       '5. Sources to corroborate the impact', 'Main panel name'],
      dtype='object')
